<a href="https://colab.research.google.com/github/JungAh12/Everyone_TF2.0/blob/master/ModelSaveandLoad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q h5py pyyaml
!pip install -q tensorflow-gpu==2.0.0-rc1

     |████████████████████████████████| 380.5MB 40kB/s 
     |████████████████████████████████| 4.3MB 47.5MB/s 
     |████████████████████████████████| 501kB 56.7MB/s 


In [2]:
import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0-rc1'

In [3]:
# data load
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28)/255.0
test_images = test_images[:1000].reshape(-1, 28*28)/255.0

11493376/11490434 [==============================] - 0s 0us/step


#define model

In [0]:
def create_model():
    model = keras.Sequential([
                              keras.layers.Dense(512, activation='relu', input_shape=(784,)),
                              keras.layers.Dropout(0.2),
                              keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss = 'sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [10]:
model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#save checkpoint

In [11]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(train_images, train_labels, epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # 훈련 단계에 콜백 전달

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 960/1000 [===========================>..] - ETA: 0s - loss: 1.1828 - accuracy: 0.6625
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 850us/sample - loss: 1.1553 - accuracy: 0.6690 - val_loss: 0.7228 - val_accuracy: 0.7840
Epoch 2/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.4408 - accuracy: 0.8858
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 308us/sample - loss: 0.4315 - accuracy: 0.8890 - val_loss: 0.5277 - val_accuracy: 0.8340
Epoch 3/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.2726 - accuracy: 0.9342
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 305us/sample - loss: 0.2813 - accuracy: 0.9310 - val_loss: 0.4563 - val_accuracy: 0.8580
Epoch 4/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.1927 - accuracy: 0.9583
Epoch

In [12]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [15]:
model = create_model()  #훈련되지 않은 모델

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도 : {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.3616 - accuracy: 0.1280
훈련되지 않은 모델의 정확도 : 12.80%


#체크포인트에서 가중치 로드하고 evaluation

In [16]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("로드한 모델 정확도 : {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.3595 - accuracy: 0.8710
로드한 모델 정확도 : 87.10%


#checkpoint callback method

콜백함수는 몇 가지 매개변수를 제공한다.

체크포인트 이름을 고유하게 만들거나 체크포인트 주기를 조정할 수 있다.

모델 훈련하고 5번 에포크마다 고유한 이름으로 체크포인트 저장.

In [45]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weight_only=True, period=5
)   #period 몇번째마다 저장할 것인지
#save_freq 사용하라는 경고가 뜨는데 save_freq=5하면 5단위로 저장안된다.

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels, epochs=50, callbacks=[cp_callback],validation_data=(test_images, test_labels),verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt
INFO:tensorflow:Assets written to: training_2/cp-0005.ckpt/assets

Epoch 00010: saving model to training_2/cp-0010.ckpt
INFO:tensorflow:Assets written to: training_2/cp-0010.ckpt/assets

Epoch 00015: saving model to training_2/cp-0015.ckpt
INFO:tensorflow:Assets written to: training_2/cp-0015.ckpt/assets

Epoch 00020: saving model to training_2/cp-0020.ckpt
INFO:tensorflow:Assets written to: training_2/cp-0020.ckpt/assets

Epoch 00025: saving model to training_2/cp-0025.ckpt
INFO:tensorflow:Assets written to: training_2/cp-0025.ckpt/assets

Epoch 00030: saving model to training_2/cp-0030.ckpt
INFO:tensorflow:Assets written to: training_2/cp-0030.ckpt/assets

Epoch 00035: saving model to training_2/cp-0035.ckpt
INFO:tensorflow:Assets written to: training_2/cp-0035.ckpt/assets

Epoch 00040: saving model to training_2/cp-0040.ckpt
INFO:tensorflow:Assets written to: training_2/cp-0040.ckpt/assets

Epoch 00045: saving model to training_2

In [50]:
!ls {checkpoint_dir}
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest  #왜 마지막이 선택이 안되는 거지..?

checkpoint			  cp-0010.ckpt	cp-0030.ckpt  cp-0050.ckpt
cp-0000.ckpt.data-00000-of-00001  cp-0015.ckpt	cp-0035.ckpt
cp-0000.ckpt.index		  cp-0020.ckpt	cp-0040.ckpt
cp-0005.ckpt			  cp-0025.ckpt	cp-0045.ckpt


'training_2/cp-0000.ckpt'

In [51]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원 모델 정확도 : {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.2585 - accuracy: 0.1340
복원 모델 정확도 : 13.40%


#수동 가중치 저장

In [52]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels, verbose=2)
print("로드된 모델 정확도 : {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.2585 - accuracy: 0.1340
로드된 모델 정확도 : 13.40%


#모델 전체 저장하기
가중치, 모델구성, 옵티마이저 설정까지 포함해서 저장한다.

In [53]:
#keras는 HDF5표준을 따르는 기본 저장 포맷을 제공한다.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 539us/sample - loss: 1.1647 - accuracy: 0.6590
Epoch 2/5
1000/1000 [==============================] - 0s 189us/sample - loss: 0.4018 - accuracy: 0.8910
Epoch 3/5
1000/1000 [==============================] - 0s 197us/sample - loss: 0.3110 - accuracy: 0.9060
Epoch 4/5
1000/1000 [==============================] - 0s 185us/sample - loss: 0.2111 - accuracy: 0.9520
Epoch 5/5
1000/1000 [==============================] - 0s 190us/sample - loss: 0.1569 - accuracy: 0.9630


In [54]:
#load
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [55]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("로드 모델 정확도 : {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5785 - accuracy: 0.8570
로드 모델 정확도 : 85.70%
